Using the methods of Colombo et al 2017 to get joint p-values from my single-experiment DESeq results for the RNA-seq experiments and call a group of NMD target transcripts

In [1]:
#combine adjusted p values into a single dataframe
import os
import pandas as pd
data_file_names = [file_name for file_name in os.listdir('DESeq_results/') if file_name.endswith('.txt') and not file_name.startswith('.')]
dfs = []
for data_file_name in data_file_names:
    data_file = os.path.join('DESeq_results/', data_file_name)
    df = pd.read_csv(data_file, sep='\t')
    df = df[['Unnamed: 0', 'padj', 'log2FoldChange']]
    df.rename(columns = {'Unnamed: 0':'tx_id', 'padj':'%s padj' % data_file_name[:-4], 'log2FoldChange':'%s log2FoldChange' % data_file_name[:-4]}, inplace = True)
    dfs.append(df)
combined_padj_df = reduce(lambda x, y: pd.merge(x, y, on = 'tx_id', how='outer'), dfs)
combined_padj_df.to_csv('all_padj_log2FC.tsv', sep='\t')

In [2]:
from scipy.special import comb
import math

def sum_of_p_values(p_values):
    '''
    Uses the sum of p values method (Edgington 1972, Journal of Psychology)
    To produce a joint p value from a list of p values
    '''
    n = len(p_values)
    s = sum(p_values)
    n_factorial = math.factorial(n)
    summation_terms = [((-1)**i)*comb(n, i)*((s-i)**n) for i in range(n) if (s-i)>0]
    new_p = sum(summation_terms)/n_factorial
    return new_p

def sum_of_2_p_values(p1, p2):
    return sum_of_p_values([p1, p2])

def sum_of_p_values_kd_rescue(kd_p, kd_l2fc, res_p, res_log2FC):
    """
    if kd_l2fc is < 0, set kd_p to 1
    if res_log2FC is > 0, set res_p to 1
    then run sum_of_p_values on the result
    """
    
    if kd_l2fc < 0:
        new_kd_p = 1
    else:
        new_kd_p = kd_p
    if res_log2FC > 0:
        new_res_p = 1
    else:
        new_res_p = res_p
    
    return sum_of_p_values([new_kd_p, new_res_p])

In [3]:
#combine p values from knockdown and rescues
import numpy as np
combined_padj_df = combined_padj_df.dropna()
meta_p = pd.DataFrame(combined_padj_df['tx_id'])
kd_rescue_pairs = [('UPF1_kd_vs_scr', 'UPF1_rescue_vs_kd', 'UPF1'),('SMG6_kd_vs_scr', 'SMG6_rescue_vs_kd', 'SMG6'), 
                  ('SMG7_kd_vs_scr', 'SMG7_rescue_vs_dSMG_kd', 'SMG7'), ('dSMG_kd_vs_scr', 'SMG6_rescue_vs_dSMG_kd', 'dKD_SMG6'),
                   ('dSMG_kd_vs_scr', 'SMG7_rescue_vs_dSMG_kd', 'dKD_SMG7')]
for pair in kd_rescue_pairs:
    meta_p[pair[2]] = np.vectorize(sum_of_p_values_kd_rescue)(combined_padj_df['%s padj'%pair[0]], 
                                                              combined_padj_df['%s log2FoldChange'%pair[0]],
                                                              combined_padj_df['%s padj'%pair[1]], 
                                                              combined_padj_df['%s log2FoldChange'%pair[1]])

In [4]:
"The results from SMG6 and dKD_SMG6 were combined with Fisher’s method in a single meta_SMG6 score."
import scipy.stats as stats
import numpy as np

def fishers_method(p1, p2):
    pvalues = [p1, p2]
    return stats.combine_pvalues(pvalues, method='fisher')[1]

meta_p['meta_SMG6'] = np.vectorize(fishers_method)(meta_p['SMG6'], meta_p['dKD_SMG6'])
meta_p['meta_SMG7'] = np.vectorize(fishers_method)(meta_p['SMG7'], meta_p['dKD_SMG7'])

In [5]:
"A meta_SMGs significance score was then computed with a sum ofP-value from meta_SMG6 and meta_SMG7"
meta_p['meta_SMGs'] = np.vectorize(sum_of_2_p_values)(meta_p['meta_SMG6'], meta_p['meta_SMG7'])

In [6]:
"""
The final significance parameter used to determine the list 
of most significant NMD targets was calculated with a Fisher’s method from meta_SMGs and UPF1_FDR (meta_meta).
"""
meta_p['meta_meta'] = np.vectorize(fishers_method)(meta_p['meta_SMGs'], meta_p['UPF1'])

In [7]:
import sys

pipeline_dir = '../ribo_seq/'
if pipeline_dir not in sys.path:
    sys.path.append(pipeline_dir)


import ribo_utils

comprehensive_gtf_data = ribo_utils.gtf_data('../annotations/orfquant_stops_collapsed.gtf')

In [8]:
def get_nmd_status(x):
    if '|' in x:
        return any([get_nmd_status(tx) for tx in x.split('|')])
    return sorted(comprehensive_gtf_data.transcript_to_entries[x])[0].get_value('transcript_type') == 'nonsense_mediated_decay'

def get_nmd_status_all(x):
    if '|' in x:
        return all([get_nmd_status(tx) for tx in x.split('|')])
    return sorted(comprehensive_gtf_data.transcript_to_entries[x])[0].get_value('transcript_type') == 'nonsense_mediated_decay'


def get_gene_id(x):
    if '|' in x:
        return get_gene_id(x.split('|')[0])
    return comprehensive_gtf_data.tx_to_genes[x]

def get_gene_name(x):
    if '|' in x:
        return get_gene_name(x.split('|')[0])
    return comprehensive_gtf_data.tx_to_gene_names[x]

In [9]:
meta_p['gene_name'] = meta_p['tx_id'].apply(get_gene_name)
meta_p['gene_id'] = meta_p['tx_id'].apply(get_gene_id)

#meta_p['tx_id'] = meta_p['Transcript'].apply(extract_tx_id)
#meta_p['gene_id'] = meta_p['Transcript'].apply(extract_gene_id)
meta_p['GENCODE nmd annotation'] = meta_p['tx_id'].apply(get_nmd_status)
#meta_p['GENCODE nmd annotation all'] = meta_p['tx_id'].apply(get_nmd_status_all)

len(meta_p[meta_p['meta_meta']<0.01])

2159

In [10]:
meta_p.to_csv('colombo_NMD_meta_pvalues.tsv', sep='\t', index=False)